In [17]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from src.models.functions import load_features
from scipy.spatial.distance import pdist


In [39]:
df = load_features('../../Data/preprocessed_cluster/features')
#df = pd.read_csv('/Users/michal/Documents/PPE_Project/Data/preprocessed_cluster/features/features_part_1.csv')

In [40]:
df

,movieId,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,165747,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,-0.5110
1,165749,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.2966
2,165753,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,-2.0500
3,165755,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,-1.5500
4,165757,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1.3570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87580,225551,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,-3.8570
87581,225553,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-3.8200
87582,225555,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,-3.8200
87583,225557,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,-3.8200


In [41]:
x = df.copy()
clustering = DBSCAN(eps=3, min_samples=2).fit(x)

In [46]:
# if x is your full DataFrame including non‐feature columns:
feature_cols = x.columns.difference(['movieId'])

# Option A: using pandas
q = (
    x
    .loc[800, feature_cols]         # or x.iloc[0][feature_cols]
    .to_numpy()                   # get a 1D numpy array
    .astype('float32')            # Annoy/FAISS want float32
)


In [47]:
import numpy as np
from annoy import AnnoyIndex

# 1) pull out only your numeric feature columns (drop movieId or any non-features)
feature_cols = df.columns.difference(['movieId'])
X = df[feature_cols].to_numpy().astype('float32')

# 2) make your Annoy index
f = X.shape[1]                # should be 23
index = AnnoyIndex(f, 'euclidean')

# 3) add each row
for i, vec in enumerate(X):
    index.add_item(i, vec)

# 4) build your forest
n_trees = 50
index.build(n_trees)

# 5) (optional) keep a mapping back to movieId
id_map = dict(enumerate(df['movieId'].tolist()))

# 6) query with a 23-dim vector q
q = q.astype('float32')       # make sure it’s float32 and length==23
idxs, dists = index.get_nns_by_vector(q, 5, include_distances=True)

# 7) map back to movieId or whatever
nearest_movie_ids = [ id_map[i] for i in idxs ]
print(nearest_movie_ids, dists)


[165747] [3.6685454845428467]
